##Pulse Rate Algorithm

### Dataset
 **Troika**[1] dataset 

Zhilin Zhang, Zhouyue Pi, Benyuan Liu, ‘‘TROIKA: A General Framework for Heart Rate Monitoring Using Wrist-Type Photoplethysmographic Signals During Intensive Physical Exercise,’’IEEE Trans. on Biomedical Engineering, vol. 62, no. 2, pp. 522-531, February 2015. Link

-----

### Code

In [ ]:
import glob

import numpy as np
import scipy as sp
import scipy.io
import scipy.signal


def LoadTroikaDataset():
    """
    Retrieve the .mat filenames for the troika dataset.

    Review the README in ./datasets/troika/ to understand the organization of the .mat files.

    Returns:
        data_fls: Names of the .mat files that contain signal data
        ref_fls: Names of the .mat files that contain reference data
         and  are ordered correspondingly, so that ref_fls[5] is the 
            reference data for data_fls[5], etc...
    """
    data_dir = "./datasets/troika/training_data"
    data_fls = sorted(glob.glob(data_dir + "/DATA_*.mat"))
    ref_fls = sorted(glob.glob(data_dir + "/REF_*.mat"))
    return data_fls, ref_fls

def LoadTroikaDataFile(data_fl):
    """
    Loads and extracts signals from a troika data file.

    Usage:
        data_fls, ref_fls = LoadTroikaDataset()
        ppg, accx, accy, accz = LoadTroikaDataFile(data_fls[0])

    Args:
        data_fl: (str) filepath to a troika .mat file.

    Returns:
        numpy arrays for ppg, accx, accy, accz signals.
    """
    data = sp.io.loadmat(data_fl)['sig']
    return data[2:]


def AggregateErrorMetric(pr_errors, confidence_est):
    """
    Computes an aggregate error metric based on confidence estimates.

    Computes the MAE at 90% availability. 

    Args:
        pr_errors: a numpy array of errors between pulse rate estimates and corresponding 
            reference heart rates.
        confidence_est: a numpy array of confidence estimates for each pulse rate
            error.

    Returns:
        the MAE at 90% availability
    """    
    # Higher confidence means a better estimate. The best 90% of the estimates
    #    are above the 10th percentile confidence.
    
    percentile90_confidence = np.percentile(confidence_est, 10)

    # Find the errors of the best pulse rate estimates
    best_estimates = pr_errors[confidence_est >= percentile90_confidence]
    

    # Return the mean absolute error
    return np.mean(np.abs(best_estimates))

def Evaluate():
    """
    Top-level function evaluation function.

    Runs the pulse rate algorithm on the Troika dataset and returns an aggregate error metric.

    Returns:
        Pulse rate error on the Troika dataset. See AggregateErrorMetric.
    """
    # Retrieve dataset files
    data_fls, ref_fls = LoadTroikaDataset()
    errs, confs = [], []
    for data_fl, ref_fl in zip(data_fls, ref_fls):
        # Run the pulse rate algorithm on each trial in the dataset
        errors, confidence = RunPulseRateAlgorithm(data_fl, ref_fl)
        errs.append(errors)
        confs.append(confidence)
        # Compute aggregate error metric
    errs = np.hstack(errs)
    confs = np.hstack(confs)
    return AggregateErrorMetric(errs, confs)

def RunPulseRateAlgorithm(data_fl, ref_fl):
    """
    Calculates pulse rate estimation and confidence 

    Args:
        data_fl: (str) filepath to a troika .mat file with measurement data.
        ref_fl: (str) filepath to a troika .mat file with reference pulse rate
    Returns:
        errors: numpy array, difference between pulse rate estimates and ground-truth value (reference)
        confidence: numpy array, confidence for each heart rate
    """
    fs = 125
    
    # Load data using LoadTroikaDataFile
    ppg, acc_x, acc_y, acc_z = LoadTroikaDataFile(data_fl)

    # Compute pulse rate estimates and estimation confidence.
    
    # Applying Bandpass filter to ppg and acc signals
    filtered_ppg = BandpassFilter(ppg, fs)
    filtered_acc_x, filtered_acc_y, filtered_acc_z = BandpassFilter(acc_x, fs), BandpassFilter(acc_y, fs), BandpassFilter(acc_z, fs)
    
    acc = np.mean([filtered_acc_x, filtered_acc_y, filtered_acc_z], axis=0)
    
    # Ground truth 
    gt = scipy.io.loadmat(ref_fl)['BPM0'].reshape(-1)
    
    # Make prediction for heart rate, confidence and errors
    preds, confs = make_prediction(acc, ppg, fs)
    
    if len(gt)>len(preds):
        gt = gt[:-1]
    
    errors = np.array(list(np.abs(np.subtract(preds, gt))))
    confidence = np.array(list(confs))
    
    return errors, confidence

def BandpassFilter(signal, fs, pass_band=(40/60.0, 240/60.0)):
    """
    Bandpass filter the signal between 40BPM and 240BPM.
    Args:
        signal: numpy array, for perioduc sugnals
        fs: int, frequency at which a sensor measures a periodic signal
    
    Returns:
    
    filtered signal after appliying Bandpass filter algorythm and return 
    frequences only in 40-240 BPM range.
    """
    b, a = sp.signal.butter(3, (40/60.0, 240/60.0), btype='bandpass', fs=fs)
    return sp.signal.filtfilt(b, a, signal)

def F_transform(sig, fs):
    """
    Fourier Transfrom of the given sequence
    """ 
    freqs = np.fft.rfftfreq(len(sig), 1/fs)
    fft_magnitude = np.abs(np.fft.rfft(sig))
    return (freqs, fft_magnitude)

def conf_fun(freqs, fft, freq):
    """
    Calculate the confidence of transformed data inputs.
    """ 
    window = (freqs > freq - 1) & (freqs < freq + 1)  
    confidence = np.sum(fft[window]) / np.sum(fft)
    return confidence

def make_prediction(acc, ppg, fs, window_len_s = 8, window_shift_s = 2):
    """
    Predict heart rate and calculate confidence using ppg and accelerometer data
    """
    
    min_bandpass = 40/60.0
    max_bandpass = 240/60.0
     
    window_length = window_len_s * fs
    window_shift = window_shift_s * fs
    
    pred = []
    confidences = []
     
    filtered_signals = {'acc':acc, 'ppg':ppg}
    
    #indices of the startig points
    indices = np.arange(start=0, stop=len(ppg) - window_length, step=window_shift)
    
    # Calculations with shifting windows
    for i in indices:
        
        window, freqs, freq, fft = {}, {}, {}, {}
        # For each signal we calculate the estimation for each window, apply Fourier transformation and filter out frequencies
        for name, s in filtered_signals.items():
            
            window[name] = s[i:i+window_length]
            freqs[name], fft[name] = F_transform(window[name], fs)
            fft[name][(freqs[name] <= min_bandpass) | (freqs[name] >= max_bandpass)] = 0.0
#           Pick frequency with the largerst magnitude
            freq[name] = freqs[name][np.argmax(fft[name], axis=0)] 
        
        ppg_freq, ppg_fft, ppg_freqs = freq['ppg'], fft['ppg'], freqs['ppg']
        acc_freq = freq['acc']
        
        freq = ppg_freq
        conf = conf_fun(ppg_freqs, ppg_fft, ppg_freq)
        
        # New estimation for second strongest frequency.
        if abs(ppg_freq - acc_freq) == 0:
            
            # Frequency with the second largerst FFT
            sec_freq = ppg_freqs[np.argsort(ppg_fft, axis=0)[-2]]
            sec_conf = conf_fun(ppg_freqs, ppg_fft, sec_freq)
            
            # Pick the larger freq
            if sec_conf > conf:
                freq, conf = sec_freq, sec_conf
        
        pred.append(freq * 60)
        confidences.append(conf)
    
     
    return pred, confidences


In [ ]:
Evaluate()

15.73826625752786

Code Description

Python libraries were used: glob, NumPy, scipy.
The code provided below estimates the pulse rate from the PPG signal and a 3-axis accelerometer.
As an output, we get an aggregate error metric based on confidence estimates.
All signals are filtered by the band pass algorithm, the 40-240BPM range was used for creating the pass band.
When the dominant accelerometer frequency equals the PPG, we pick the second-strongest one.



Data Description

The dataset comes from a biomedical paper: Z. Zhang, Z. Pi, B. Liu, TROIKA: A general framework for heart rate monitoring using wrist-type photoplethysmographic signals during intensive physical exercise. 

PPG signals were recorded from wristband tracker among people from 18 to 35 years old. The acceleration signal was also recorded from the wrist by a three-axis accelerometer during different physical activities: resting, walking, and running. All signals were sampled at 125Hz and sent via Bluetooth to the computer.

Possible shortcomings:
As all participants were males, so there could be a hidden bias. In addition the data were obtained only from young people, to be more confident in our algorithm it should be tested on a wider range including old people and kids/teenagers. 


Algorithm Description

The algorithm estimates the pulse rate from the PPG signal. The loaded signal values are filtered using the Bandpass algorithm. The 40-240BPM range was used for creating a pass band, in addition, Fourier transformation was applied. We compute the largest FFT coefficient, in case the dominant accelerometer frequency is equal to the PPG, we pick the second strongest frequency as the best estimate. To be precise: after getting predicion we compare it with acc signals to be confident that this value doesn't come from the noise (movement).

As an output of the model, we are getting the MAE (mean absolute error) of the prediction and ground-truth heart rate array and array of confidence scores. A measure of confidence is based on how much energy in the frequency spectrum is concentrated near an estimation of pulse rate. Based on the confidence level, we can evaluate whether the estimated pulse rate is accurate.

Common failure models: Shouldn't be applied without fine-tuning to the other datasets that are not very similar to the given one.

Algorithm Performance

Algorithm performs quite well: MAE for training is 15.72, on the test: 9.13
It performed even better on test data than on the train, but this is not common in general, potentially there data in the test dataset was too simple compared to the train data.




-----
### Next Steps
You will now go to **Test Your Algorithm** to apply a unit test to confirm that your algorithm met the success criteria. 